<a href="https://colab.research.google.com/github/Gaurav7004/RSE_TOOL/blob/main/RSE_TOOL_11thApril2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
# ! pip install pandarallel
# from pandarallel import pandarallel
import json
import re

In [2]:
# pandarallel.initialize()

In [3]:
df = pd.read_csv('Data1.csv')

# remove blank spaces from front and back of column names
df.columns = df.columns.str.strip()
df.columns

<ipython-input-3-fc8b3e05b5c4>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data1.csv')


Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG'],
      dtype='object')

In [4]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('Example1.RSE', "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

{
    "#SAMPLING SUBSAMPLE2": [],
    "#GROUP 2": [
        "SEC   C sec",
        "ST_GR C NEWVAR"
    ],
    "#FILTER 1": [
        "AGE>=15"
    ],
    "#VARIABLE 3  ": [
        "POP\tN  NEWVAR",
        "LF\tN  NEWVAR",
        "WRK\tN  NEWVAR"
    ],
    "#FILE Data1.csv  ": [],
    "#RENAME  5    ": [
        "STRMID C sec+st+strm+sstrm",
        "SS     C ss",
        "NSS    N  nss",
        "NSC    N  nsc",
        "MULT   N  mult"
    ],
    "#TRANSFORM 4  ": [
        "ST_GR=(\"G1\")   ST in (\"10,19,20,21\")",
        "ST_GR=(\"G2\")   ST in (\"11,12,13,14,15,16,17,18\")",
        "ST_GR=(\"G3\")   Otherwise",
        "POP=(1)",
        "WRK=(1)  CWS in (\"11,12,21,31,41,51,61,61,71,72\")",
        "WRK=(0)  CWS NOT in (\"11,12,21,31,41,51,61,61,71,72\")",
        "LF=(1)      CWS in (\"11,12,21,31,41,51,61,61,71,72,81\")",
        "LF=(0)      Otherwise"
    ],
    "#EST_RSE 7": [
        "S POP",
        "E POP",
        "R POP",
        "E 100*LF/POP",
        "R LF/POP

In [5]:
def SAMPLING_(data, df):
    for key, value in data.items():
        pass

def GROUP_(data, df):
    for key, value in data.items():
        ### To read the sampling method used
        if str('SAMPLING').lower() in str(key).lower():
            if str('SUBSAMPLE2').lower() in str(key).lower():
                return str('SUBSAMPLE2')
            elif str('SRSWR').lower() in str(key).lower():
                pass
            elif str('SRSWOR').lower() in str(key).lower():
                pass

def FILTER_(data, df):
    for key, value in data.items():
        ### To read the sampling method used
        if str('FILTER').lower() in str(key).lower():
            ### Replace logical AND '&&' with 'and'
            if '&&' in value[0]:
                return value[0].replace('&&', '&')
            ### Replace logical OR '||' with 'or'
            elif '||' in value[0]:
                return value[0].replace('||', '|')
            else:
                return value[0]

def FILE_(data):
    for key, value in data.items():
        ### To read the file used
        if str('FILE').lower() in str(key).lower():
            if key.strip().endswith('.xlsx') or key.strip().endswith('.csv'):
                match = re.search(r'[A-Za-z0-9]+\.xlsx|[A-Za-z0-9]+\.csv', key)   
                if match:
                    file_name = match.group(0)
                    return file_name
                else:
                    file_name = None
                    return file_name
                print(file_name)

            # return file_name

def RENAME_(data, df):
    # create an empty dictionary to store the column names
    column_dict = {}

    for key, value in data.items():
        ### To read the file used
        if str('RENAME').lower() in str(key).lower():
            # value = [elem for elem in value[0].split(' ') if elem.strip()]
            # return value

            for col in value:
                col_split = col.split()

                # print(col_split[0])

                # extract the last element as the column name
                col_name = col_split[-1]

                new_col_name = col_split[0]

                ### if there is a "+" sign in the last element, concatenate the columns
                if "+" in col_name:
                    col_concat = col_name.split("+")
                    col_concat = [i.upper() for i in col_concat]

                    # concatenate the columns
                    df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)

                else:
                    col_name = col_name.upper()
                    print(type(col_name))


def NEWVARIABLE_(data, df):
    for key, value in data.items():
        ### To read the file used
        if str('VARIABLE').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            
            for name in value:
                df[name] = 0


def TRANSFORM_(data):
    global condition

    # Initialize empty arrays to store the values
    Arr1 = []
    Arr2 = []
    Arr3 = []

    for key, value in data.items():
        ### To read the file used
        if str('TRANSFORM').lower() in str(key).lower():
            lines = '\n'.join(value)
            lines = lines.split("\n")

            # Loop through the lines and extract the values
            for line in lines:
                line_values = line.split('=')  # split the line into values
                
                var_name = line_values[0].strip()  # extract the variable name
                
                var_value = line_values[1].strip().split()[0]  # extract the variable value
                
                # Regular expression pattern to extract the condition
                pattern = r'^\w+=\([^\)]*\)(?:\s+(.*?)(?:\n|$))?$'

                # Split the string into lines and extract the condition from each line
                conditions = []

                for line in line.split('\n'):
                    match = re.match(pattern, line)
                 
                    if match:
                        condition = match.group(1)
                        
                        if condition is not None:
                            conditions.append(condition.strip())
                        else:
                            conditions.append(None)

                # Append the values to the arrays
                Arr1.append(var_name)
                Arr2.append(var_value)
                Arr3.append(condition)

    return Arr1, Arr2, Arr3
        

def EST_RSE_(data):
    for key, value in data.items():
        pass

TRANSFORM_(data)[2]

['ST in ("10,19,20,21")',
 'ST in ("11,12,13,14,15,16,17,18")',
 'Otherwise',
 None,
 'CWS in ("11,12,21,31,41,51,61,61,71,72")',
 'CWS NOT in ("11,12,21,31,41,51,61,61,71,72")',
 'CWS in ("11,12,21,31,41,51,61,61,71,72,81")',
 'Otherwise']

In [6]:
def parse_query_string(query_string):
    # Replace curly quotes with straight quotes
    query_string = query_string.replace("”", "\"")
    
    if 'in' in query_string:
        if ('NOT in' or 'not in') in query_string:
            query_string = re.sub(r'(\b\w+\b)\s+NOT in\s*\(([^)]+)\)', r'~\1.isin([\2])', query_string)
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        elif 'in' in query_string:
            # Replace 'in' operators with 'isin' and add list brackets
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        else:
            pass
            
    else:
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)('[^']*'|"[^"]*")|(\b\w+\b)(\s*[<>=!]+\s*)("[^]*'|"[^"]*")''', r'\1\2\3', query_string)
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)(\d+)''', r'\1\2\3', query_string)

    # Replace double quotes with single quotes
    query_string = query_string.replace("\"", "'")
    
    # Replace 'AND' with '&', 'OR' with '|', and add parentheses
    query_string = query_string.replace(" AND ", " & ")\
                        .replace(" OR ", " | ").\
                            replace("&&", '&').\
                                replace('||', '|').\
                                    replace('<>', '!=')
    
    # Replace column names with df[column] syntax
    query_string = re.sub(r'(\b\w+\b)=', r'df["\1"]==', query_string)


    # Define a function to replace variable names with df["<variable_name>"]
    def replace_var(match):
        return f'df["{match.group(0)}"]'

    # Use regex to replace variable names with df["<variable_name>"]
    pattern = re.compile(r'''\b\w+\b(?=\s*[<>=!]=*\s*[\'"\d])''')
    new_string = pattern.sub(replace_var, query_string)

    query_string = re.sub(r"'(\d+)'|\"(\d+)\"", lambda match: str(int(match.group(1) or match.group(2))), query_string)
    query_string = re.sub(r'\[(\d+,\s*)+\d+\]', lambda match: '[' + match.group(0).replace("\'", "'") + ']', query_string)
    
    return query_string


final_parsed_query = []

for i in list(TRANSFORM_(data))[2]:
    # print(i)
    if i:
        parsed_query = parse_query_string(i)
        final_parsed_query.append(parsed_query)
    else:
        final_parsed_query.append(i)

# new_lst = [s.replace("['", "[").replace("']", "]") for s in final_parsed_query]
# print([s.replace("['", "[").replace("']", "]") for s in final_parsed_query if s else pass])
# final_parsed_query = [s.replace("['", "[").replace("']", "]") if s is not None else None for s in final_parsed_query]
final_parsed_query

["ST.isin(['10,19,20,21'])",
 "ST.isin(['11,12,13,14,15,16,17,18'])",
 'Otherwise',
 None,
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "~CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72,81'])",
 'Otherwise']

In [7]:
df.head()

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,SEX,MSEX,AGE,MARST,GEDU_LVL,PAS,PS_SS,CWS,ERN_SELF,ERN_REG
0,1,28,17469,1,1,1,1,1,1,1,...,1,1,40.0,2,1.0,51,51,51,0,0
1,1,28,17469,1,1,1,1,1,1,1,...,2,2,18.0,1,8.0,91,91,91,0,0
2,1,28,17469,1,1,1,1,1,1,1,...,2,2,19.0,1,10.0,91,91,91,0,0
3,1,28,17469,1,1,1,1,1,1,1,...,2,2,38.0,2,1.0,51,51,51,0,0
4,1,28,17469,2,1,1,2,1,1,1,...,1,1,21.0,1,5.0,51,51,51,0,0


In [8]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG'],
      dtype='object')

In [9]:
Arr1 = list(TRANSFORM_(data))[0]
Arr2 = list(TRANSFORM_(data))[1]
Arr2 = [i.strip('()\'\"') for i in Arr2]
Arr3 = final_parsed_query

print(Arr1)
print(Arr2)
print(Arr3)

['ST_GR', 'ST_GR', 'ST_GR', 'POP', 'WRK', 'WRK', 'LF', 'LF']
['G1', 'G2', 'G3', '1', '1', '0', '1', '0']
["ST.isin(['10,19,20,21'])", "ST.isin(['11,12,13,14,15,16,17,18'])", 'Otherwise', None, "CWS.isin(['11,12,21,31,41,51,61,61,71,72'])", "~CWS.isin(['11,12,21,31,41,51,61,61,71,72'])", "CWS.isin(['11,12,21,31,41,51,61,61,71,72,81'])", 'Otherwise']


In [10]:
df.head(3)

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,SEX,MSEX,AGE,MARST,GEDU_LVL,PAS,PS_SS,CWS,ERN_SELF,ERN_REG
0,1,28,17469,1,1,1,1,1,1,1,...,1,1,40.0,2,1.0,51,51,51,0,0
1,1,28,17469,1,1,1,1,1,1,1,...,2,2,18.0,1,8.0,91,91,91,0,0
2,1,28,17469,1,1,1,1,1,1,1,...,2,2,19.0,1,10.0,91,91,91,0,0


In [11]:
set(Arr1) - set(list(df.columns))

{'LF', 'POP', 'ST_GR', 'WRK'}

In [12]:

# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)

new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

mismatched_names

{'LF', 'POP', 'ST_GR', 'WRK'}

In [13]:
Arr3

["ST.isin(['10,19,20,21'])",
 "ST.isin(['11,12,13,14,15,16,17,18'])",
 'Otherwise',
 None,
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "~CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72,81'])",
 'Otherwise']

In [14]:
Arr3 = [i.replace("(['", "([").replace("'])", "])") if i is not None else 'True' if Arr3[j] == 'Otherwise' else True for j, i in enumerate(Arr3)]
Arr3

['ST.isin([10,19,20,21])',
 'ST.isin([11,12,13,14,15,16,17,18])',
 'Otherwise',
 True,
 'CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 '~CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 'CWS.isin([11,12,21,31,41,51,61,61,71,72,81])',
 'Otherwise']

In [15]:
new_arr3 = []
for i, value in enumerate(Arr3):
    if value == 'Otherwise':
        other_values = set([Arr3[j] for j in range(len(Arr3)) if Arr1[j] == Arr1[i] and Arr3[j] != 'Otherwise'])
        new_value = '~' + ' & ~'.join(other_values)
        new_arr3.append(new_value)
    else:
        new_arr3.append(value)

new_arr3

['ST.isin([10,19,20,21])',
 'ST.isin([11,12,13,14,15,16,17,18])',
 '~ST.isin([11,12,13,14,15,16,17,18]) & ~ST.isin([10,19,20,21])',
 True,
 'CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 '~CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 'CWS.isin([11,12,21,31,41,51,61,61,71,72,81])',
 '~CWS.isin([11,12,21,31,41,51,61,61,71,72,81])']

In [16]:
Arr3 = new_arr3

In [17]:
# Apply the queries in Arr3 to filter the rows of the dataframe
for i, query_str in enumerate(Arr3):
    if query_str is True:
        df[Arr1[i]] = Arr2[i]
    else:
        df_query = df.query(query_str)
        df.loc[df_query.index, Arr1[i]] = Arr2[i]

print(df)

        SEC  ST    FSU  IDNO  SEG  SSS  SSU  STRM  SSTRM  SS  ...  GEDU_LVL  \
0         1  28  17469     1    1    1    1     1      1   1  ...       1.0   
1         1  28  17469     1    1    1    1     1      1   1  ...       8.0   
2         1  28  17469     1    1    1    1     1      1   1  ...      10.0   
3         1  28  17469     1    1    1    1     1      1   1  ...       1.0   
4         1  28  17469     2    1    1    2     1      1   1  ...       5.0   
...     ...  ..    ...   ...  ...  ...  ...   ...    ...  ..  ...       ...   
109851    2  34  27428     7    1    4    1     1      3   2  ...       1.0   
109852    2  34  27428     8    1    4    2     1      3   2  ...       1.0   
109853    2  34  27428     8    1    4    2     1      3   2  ...       5.0   
109854    2  34  27428     8    1    4    2     1      3   2  ...       6.0   
109855    2  34  27428     8    1    4    2     1      3   2  ...       1.0   

        PAS  PS_SS  CWS  ERN_SELF  ERN_REG  WRK  PO

In [18]:
df['ST_GR'].unique()

array(['G3', 'G2', 'G1'], dtype=object)

In [19]:
df['POP'].unique()

array(['1'], dtype=object)

In [20]:
df.to_csv('WorkFile_Balanced_Half_Sample_Method.csv')